In [2]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [15]:
%matplotlib inline
import matplotlib.pyplot as plt

In [4]:
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10)) # wave 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + wave 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5) # + noise
    return series[..., np.newaxis].astype(np.float32)

In [5]:
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

In [6]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(25, return_sequences = True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(25),
    keras.layers.Dense(1)
])

In [7]:
model.compile(loss="mse", optimizer = 'adam', metrics = [tf.keras.metrics.MeanSquaredError()])

In [8]:
history = model.fit(X_train, y_train, epochs = 10, validation_data = (X_valid, y_valid))

Train on 7000 samples, validate on 2000 samples
Epoch 1/10
7000/7000 [==============================] - 9s 1ms/sample - loss: 0.0433 - mean_squared_error: 0.0433 - val_loss: 0.0065 - val_mean_squared_error: 0.0065
Epoch 2/10
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0052 - mean_squared_error: 0.0052 - val_loss: 0.0041 - val_mean_squared_error: 0.0041
Epoch 3/10
7000/7000 [==============================] - 7s 1ms/sample - loss: 0.0038 - mean_squared_error: 0.0038 - val_loss: 0.0033 - val_mean_squared_error: 0.0033
Epoch 4/10
7000/7000 [==============================] - 7s 1ms/sample - loss: 0.0033 - mean_squared_error: 0.0033 - val_loss: 0.0030 - val_mean_squared_error: 0.0030
Epoch 5/10
7000/7000 [==============================] - 7s 1ms/sample - loss: 0.0031 - mean_squared_error: 0.0031 - val_loss: 0.0029 - val_mean_squared_error: 0.0029
Epoch 6/10
7000/7000 [==============================] - 7s 1ms/sample - loss: 0.0030 - mean_squared_error: 0.0030 - val_lo

In [12]:
series = generate_time_series(1, n_steps + 10)
X_new, Y_new = series[:, :n_steps], series[:, n_steps:]
X = X_new
for step_ahead in range(10):
    y_pred_one = model.predict(X[:, step_ahead:])[:, np.newaxis, :]
    X = np.concatenate([X, y_pred_one], axis=1)
Y_pred = X[:, n_steps:]

In [37]:
Y_pred

array([[[0.5306229 ],
        [0.63327557],
        [0.65679276],
        [0.6029941 ],
        [0.48471808],
        [0.35099503],
        [0.24551986],
        [0.17174028],
        [0.10406645],
        [0.04242638]]], dtype=float32)

In [39]:
Y = np.empty((10000, n_steps, 10)) # each target is a sequence of 10D vectors
for step_ahead in range(1, 10 + 1):
    Y[:, :, step_ahead - 1] = series[:, step_ahead:step_ahead + n_steps, 0]
Y_train = Y[:7000]
Y_valid = Y[7000:9000]
Y_test = Y[9000:]

In [41]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

In [43]:
def last_time_step_mse(Y_true, Y_pred):
    return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])
optimizer = keras.optimizers.Adam(lr=0.01)
model.compile(loss="mse", optimizer=optimizer, metrics=[last_time_step_mse])

In [44]:
history = model.fit(X_train, Y_train, epochs = 10, validation_data = (X_valid, Y_valid))

Train on 7000 samples, validate on 2000 samples
Epoch 1/10
7000/7000 [==============================] - 9s 1ms/sample - loss: 0.0405 - last_time_step_mse: 0.0533 - val_loss: 0.0016 - val_last_time_step_mse: 0.0030
Epoch 2/10
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0015 - last_time_step_mse: 0.0035 - val_loss: 0.0017 - val_last_time_step_mse: 0.0044
Epoch 3/10
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0011 - last_time_step_mse: 0.0021 - val_loss: 8.5965e-04 - val_last_time_step_mse: 0.0020
Epoch 4/10
7000/7000 [==============================] - 7s 1ms/sample - loss: 0.0018 - last_time_step_mse: 0.0042 - val_loss: 0.0016 - val_last_time_step_mse: 0.0019
Epoch 5/10
7000/7000 [==============================] - 7s 1ms/sample - loss: 4.8514e-04 - last_time_step_mse: 6.3747e-04 - val_loss: 0.0011 - val_last_time_step_mse: 0.0030
Epoch 6/10
7000/7000 [==============================] - 8s 1ms/sample - loss: 5.4447e-04 - last_time_step_mse: